In [1]:
import scanpy

In [2]:
import numpy as np

In [3]:
data_files = !find 'output/datasets/predict_modality/' -type f -name '*train*h5ad'

In [4]:
data = {}
for filename in data_files:
    data[filename.split('/')[-1]] = scanpy.read_h5ad(filename)

In [5]:
atac_data = data['openproblems_bmmc_multiome_phase1_mod2.censor_dataset.output_train_mod1.h5ad']

In [8]:
from urllib.request import urlopen

In [14]:
from time import sleep

In [ ]:
from random import random

In [35]:
from xml.etree.ElementTree import fromstring

In [61]:
atacs = atac_data.var.index.to_list()

atac_seqs = {}
N = int(len(atacs)/100)

for i,region in enumerate(atacs):
    if i%N == 0:
        print(f'{i/N:.2f}')
    chrm, start, end = region.split('-')
    url = f'http://genome.ucsc.edu/cgi-bin/das/hg38/dna?segment={chrm}:{start},{end}'
    data = urlopen(url).read()
    xml = fromstring(data)
    seq = xml[0][0].text
    seq = seq.replace('\n','')
    atac_seqs[region] = seq
    sleep(.01)
    

0.00
1.00
2.00
3.00
4.00
5.00
6.00
7.00
8.00
9.00
10.00
11.00
12.00
13.00
14.00
15.00
16.00
17.00
18.00
19.00
20.00
21.00
22.00
23.00
24.00
25.00
26.00
27.00
28.00
29.00
30.00
31.00
32.00
33.00
34.00
35.00
36.00
37.00
38.00
39.00
40.00
41.00
42.00
43.00
44.00
45.00
46.00
47.00
48.00
49.00
50.00
51.00
52.00
53.00
54.00
55.00
56.00
57.00
58.00
59.00
60.00
61.00
62.00
63.00
64.00
65.00
66.00
67.00
68.00
69.00
70.00
71.00
72.00
73.00
74.00
75.00
76.00
77.00
78.00
79.00
80.00
81.00
82.00
83.00
84.00
85.00
86.00
87.00
88.00
89.00
90.00
91.00
92.00
93.00
94.00
95.00
96.00
97.00
98.00
99.00
100.00


ParseError: no element found: line 6, column 0 (<string>)

In [62]:
len(atac_seqs)

116468

In [63]:
import pickle

In [64]:
pickle.dump(atac_seqs, open('input/atac_sequences.pickle','wb'))

In [66]:
from collections import Counter

In [70]:
seq_lens = Counter()
for atac,seq in atac_seqs.items():
    seq_lens[len(seq)]+=1

seq_lens.most_common(10)

[(904, 1371),
 (903, 1348),
 (899, 1328),
 (901, 1324),
 (906, 1306),
 (900, 1293),
 (908, 1291),
 (905, 1285),
 (907, 1280),
 (895, 1267)]

In [68]:
max(seq_lens)

1741

In [69]:
min(seq_lens)

213

In [71]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [80]:
ns = 0
for atac, seq in atac_seqs.items():
    if 'n' in seq:
        ns+=1
print(ns)

22


In [81]:
chars = Counter()
for atac, seq in atac_seqs.items():
    for ch in seq:
        chars[ch]+=1
print(chars.most_common())

[('t', 24938768), ('a', 24935259), ('g', 24698723), ('c', 24643051), ('n', 1795)]


In [90]:
atac_list = []
seq_list = []
for atac, seq in list(atac_seqs.items()):
    seq = seq.replace('n','')
    seq = seq.upper()
    # Trim to length multiple of chunksize=6
    seq = seq[:len(seq)-(len(seq)%6)]
    seq = ' '.join(list(chunks(seq,6)))
    seq_list.append(seq)
    atac_list.append(atac)

In [91]:
seq

'ACTCCA TTCCAC TCCATC CCGTTC CATTCC GCTCCA TCCCAC TGCACT CCACTC CACCCC TCTCCA CTCCTC TTCCTT CATTCC ATTCCA TTCCAC CTTTTT CCATTC CTCTTG ATTGCA CTCCAA TCCACT TCACTC CACTCC ACTCCA ATCCAT TCCATT CCATTC CACTGC ATTCAA TTCCAT TCCTTT CTTTAG AGAGTA TCTCAC TCTGTC ACCAAG CCTGGA GTGCAG TGGCAC AATCTC AGCTCA CATTTC ATTTCA CCATTC CATTGC ATTCTA CTCCAT TAAATT CCACTC CACTCC ACTCCA ATCCAT TCCACT CCACTC CACTCC ATTCCA GTCCAC TCCAGT CCTTTC CATTCC ATCCCA TTCCAT TCCACT CCTTTC TACTCC ACTCCA CTGCTC TCAACT CCATTA CATTCC ATTCCA CTCCAT TCCATT CCAGTA CTTTCT TTTGAC TGTATC TCACTC TGTCAC CCAGGC TGGTGT GCAGTG GCACAA TCTCAG CTCACA TTTCAT TTCACC ATTCCA TTCCAT TCCATT CCACTG CACTGC ACTCCA CTCAAT TCCACT CCACTC TACTCC AATCCG TTCCAT TCCATT TCAATC CCTTCC ATTCCA CTCCTT TCTTTC GACAGG GTCTCC CTCTGT CACACA GGCTGT AGTGCA GGTGAT GAGACT ATTTTA TAGTCT CTGTGT ATGGGG GCATTT TGACCT ATCTCT GCACTG ATCACC CAAGTG ATGTAA CACTTG ACTATG CTCTGC CTATTG GGGCAT AGTGAC ATATCA CTGCAT TGATCA CTCAGA TGTTGT AGCTCT TGCCTA GGCTCT GCCTAC ATGGGC ATTGTG ACACA

In [92]:
with open('input/atac_list.txt','w') as atac_file:
    for atac in atac_list:
        print(atac, file=atac_file)
with open('input/atac_sequences.txt','w') as seq_file:
    for seq in seq_list:
        print(seq, file=seq_file)